# **Tarea 2 - Deep Learning - César Núñez**

## Montar Google Drive a Google Colab

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Importando packages necesarios

In [1]:
import numpy
import matplotlib.pyplot as plt
from pandas import read_csv
import math
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from tensorflow.keras.utils import plot_model
from keras.callbacks import ModelCheckpoint, EarlyStopping

# semilla aleatoria para reproducibilidad
numpy.random.seed(7)

## Conjunto de datos

### Cargando los datos
Para cargar el conjunto de datos usaremos a biblioteca Pandas. Para luego onvertir los valores enteros en valores de coma flotante que son más adecuados para modelar con una red neuronal.

In [2]:
# cargamos el conjunto de datos
dataframe = read_csv('international-airline-passengers.csv', usecols=[1], engine='python', skipfooter=3)
dataset = dataframe.values
dataset = dataset.astype('float32')

Las Redes recurrentes son sensibles a la escala de los datos de entrada, especialmente cuando se utilizan las funciones de activación sigmoide (por defecto) o tanh. Puede ser una buena práctica reescalar los datos al rango de 0 a 1, también llamado normalización

In [3]:
# normalizacion de los datos
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

### Entrenamiento y Pruebas

In [4]:
#dividimos datos en entreno y datos para el test
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print(len(train), len(test))

96 48


In [5]:
# convertir un array de valores en una matriz de conjuntos de datos
def create_dataset(dataset, look_back=1):
  dataX, dataY = [], []
  for i in range(len(dataset)-look_back-1):
    a = dataset[i:(i+look_back), 0]
    dataX.append(a)
    dataY.append(dataset[i + look_back, 0])
  return numpy.array(dataX), numpy.array(dataY)

In [6]:
# remodelamos X=t y Y=t+1
# look_back - Moficar
# LSTMs - mumero de neuronas, Cuantas Lstms, y que valores tendria sus neuronios
# Modificar la Funcion de activacion, leaning rate, optimizador (opcional)
def model_lstm(look_back = 1, optimizer = "adam", loss='mean_squared_error'):
  trainX, trainY = create_dataset(train, look_back)
  testX, testY = create_dataset(test, look_back)
  
  # cambiar la forma de la entrada para que sea[muestras, pasos de tiempo, características]
  trainX = numpy.reshape(trainX, (trainX.shape[0], trainX.shape[1],1))
  testX = numpy.reshape(testX, (testX.shape[0],testX.shape[1],1))
  
  # creacion de la red LSTM
  model = Sequential()
  #model.add(LSTM(4, input_shape=(1, look_back)))
  model.add(LSTM(8, input_shape=(look_back,1),return_sequences=True))
  model.add(LSTM(4, input_shape=(look_back,1)))
  model.add(Dense(1))
  
  
  model.compile(loss=loss, optimizer=optimizer)
  
  history = model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=0)
  ## Evaluando el Modelo
  
  # hacemos las predicciones
  trainPredict = model.predict(trainX)
  testPredict = model.predict(testX)
  
  # invertimos las predicciones
  trainPredict = scaler.inverse_transform(trainPredict)
  trainY = scaler.inverse_transform([trainY])
  testPredict = scaler.inverse_transform(testPredict)
  testY = scaler.inverse_transform([testY])
  # calculamos el error rms
  trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
  print('Resultado del entrenamiento: %.2f RMSE' % (trainScore))
  testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
  print('Resultado del test: %.2f RMSE' % (testScore))
  return trainScore, testScore

In [10]:
modelList = []
lookList = []
optmList = []
lossList = []
trainList = []
testList = []
cont_model = 1
for look in range(1,10,2):
  for optm in list(["adam","SGD", "Adagrad"]):
    for loss in list(["mean_squared_error","mean_absolute_error"]):
      modelList.append(f"Modelo N° {cont_model}")
      lookList.append(look) 
      optmList.append(optm)
      lossList.append(loss)
      trainS, testS = model_lstm(look_back = look, optimizer = optm, loss=loss)
      trainList.append(trainS)
      testList.append(testS)
      cont_model = cont_model + 1

2/2 [==============================] - 0s 0s/step
Resultado del entrenamiento: 24.64 RMSE
Resultado del test: 50.05 RMSE
2/2 [==============================] - 0s 5ms/step
Resultado del entrenamiento: 22.86 RMSE
Resultado del test: 54.67 RMSE
2/2 [==============================] - 0s 0s/step
Resultado del entrenamiento: 70.83 RMSE
Resultado del test: 214.24 RMSE
2/2 [==============================] - 0s 13ms/step
Resultado del entrenamiento: 66.96 RMSE
Resultado del test: 213.08 RMSE
2/2 [==============================] - 0s 4ms/step
Resultado del entrenamiento: 69.43 RMSE
Resultado del test: 219.71 RMSE
2/2 [==============================] - 0s 0s/step
Resultado del entrenamiento: 72.22 RMSE
Resultado del test: 229.97 RMSE
2/2 [==============================] - 0s 1ms/step
Resultado del entrenamiento: 22.26 RMSE
Resultado del test: 46.45 RMSE
2/2 [==============================] - 0s 5ms/step
Resultado del entrenamiento: 22.84 RMSE
Resultado del test: 89.72 RMSE
2/2 [=================

In [15]:
import pandas as pd
# Crear un diccionario con los datos
datos = {'Model': modelList,
         'Lookback' : lookList,
         'Optimizer' : optmList,
         'Loss function': lossList,
         'MSE Train': trainList,
         'MSE Test': testList}

# Crear un DataFrame a partir del diccionario
tabla_resumen = pd.DataFrame(datos)

resumen_lookback = tabla_resumen.groupby('Lookback').mean('MSE Test')
print(resumen_lookback)

resumen_optimizer = tabla_resumen.groupby('Optimizer').mean('MSE Test')
print(resumen_optimizer)

resumen_loss = tabla_resumen.groupby('Loss function').mean('MSE Test')
print(resumen_loss)



          MSE Train    MSE Test
Lookback                       
1         54.489602  163.618713
3         40.697036  121.033998
5         40.956976  117.841172
7         39.963407  110.527270
9         37.337581  102.158236
           Lookback  MSE Train    MSE Test
Optimizer                                 
Adagrad         5.0  60.828044  187.152192
SGD             5.0  44.390404  118.120641
adam            5.0  22.848312   63.834801
                     Lookback  MSE Train    MSE Test
Loss function                                       
mean_absolute_error       5.0  42.146181  126.177937
mean_squared_error        5.0  43.231659  119.893819


# Resultados
Como se puede observar en la tabla `resumen_look`, en promedio el error cuadrático medio es menor en la medida que aumenta el look back en el modelo, aunque la mejora en la precisión del modelo tanto en el conjunto de entrenamiento como en el conjunto de prueba. 

Como se resume en la tabla `resumen_optimizer`, en promedio el error cuadrático medio es significativamente menor cuando se usa el optimizador `adam` en comparación a cuando se utiliza `SGD` o `Adagrad`, tanto en el conjunto de entrenamiento como en el conjunto de prueba.

Como se puede observar en la tabla `resumen_loss`, en promedio el error cuadrático medio es menor cuando la medida de pérdida es el `mean_squared_error` tomando en cuanta los resultados en el conjunto de prueba a pesar de que ocurre lo contrario en los resultados del conjunto de entrenamiento. 